In [1]:
# import PyRosetta library
from pyrosetta import *
init("-newdna -beta")

# download a PDB from RCSB
from pyrosetta.toolbox import pose_from_rcsb

PyRosetta-4 2023 [Rosetta PyRosetta4.conda.m1.cxx11thread.serialization.python39.Release 2023.12+release.1d8714894eec1dac6443b64d144d220c4aed4f9c 2023-03-20T20:16:16] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init.score_function_corrections: [ WARNING ] Flag -beta_nov16 is set but either -weights are also specified or this is being used with the genpot score function.  Not changing input weights file!
core.init: Rosetta version: PyRosetta4.conda.m1.cxx11thread.serialization.python39.Release r343 2023.12+release.1d8714894ee 1d8714894eec1dac6443b64d144d220c4aed4f9c http://www.pyrosetta.org 2023-03-20T20:16:16
core.init: command: PyRosetta -newdna -beta -database /Users/tgoel/miniconda3/envs/genome/lib/python3.9/site-packages/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/uran

In [2]:
# load a DNA-containing structure
pose = pose_from_rcsb("1yrn")
pose.dump_pdb("/Users/tgoel/Downloads/1yrn_ros.pdb")

core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 985 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 0.965561 seconds.
core.import_pose.import_pose: File '1YRN.clean.pdb' automatically determined to be of type PDB
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue GLN 53
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on residue GLN 53
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OE1 on residue GLN 53
core.conformation.Conformation: [ WARNING ] missing heavyatom:  NE2 on residue GLN 53
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue ARG 56
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on residue ARG 56
core.conformation.Conformation: [ WARNING ] missing heavyatom:  NE  on residue ARG 56
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CZ  on residue ARG 56
core.conformation.Confo

True

In [3]:
pose.set_beta(2, 60.0)

In [4]:
# relax (the protein components) of a DNA-containing structure
scorefxn = rosetta.core.scoring.get_score_function()
fastrelax = rosetta.protocols.relax.FastRelax(scorefxn, 1)
fastrelax.apply(pose)
pose.dump_pdb("/Users/tgoel/Downloads/1yrn_rlx.pdb")

core.scoring.ScoreFunctionFactory: SCOREFUNCTION: beta_genpot.wts
protocols.relax.RelaxScriptManager: Reading relax scripts list from database.
protocols.relax.RelaxScriptManager: Looking for MonomerRelax2019.txt
protocols.relax.RelaxScriptManager: ================== Reading script file: /Users/tgoel/miniconda3/envs/genome/lib/python3.9/site-packages/pyrosetta/database/sampling/relax_scripts/MonomerRelax2019.txt ==================
protocols.relax.RelaxScriptManager: repeat %%nrepeats%%
protocols.relax.RelaxScriptManager: coord_cst_weight 1.0
protocols.relax.RelaxScriptManager: scale:fa_rep 0.040
protocols.relax.RelaxScriptManager: repack
protocols.relax.RelaxScriptManager: scale:fa_rep 0.051
protocols.relax.RelaxScriptManager: min 0.01
protocols.relax.RelaxScriptManager: coord_cst_weight 0.5
protocols.relax.RelaxScriptManager: scale:fa_rep 0.265
protocols.relax.RelaxScriptManager: repack
protocols.relax.RelaxScriptManager: scale:fa_rep 0.280
protocols.relax.RelaxScriptManager: min 0.01

True

In [5]:
# minimize DNA with the DNA energy function
def min_pose(pose, scorefxn, mm):
    min_mover = rosetta.protocols.minimization_packing.MinMover()
    min_mover.movemap(mm)
    min_mover.score_function(scorefxn)
    min_mover.min_type("lbfgs_armijo")
    min_mover.tolerance(1e-6)
    min_mover.apply(pose)

pose = pose_from_rcsb("1bna")

#scorefxn = rosetta.core.scoring.ScoreFunctionFactory.create_score_function("dna.wts")
scorefxn = rosetta.core.scoring.get_score_function()
scorefxn.set_weight(rosetta.core.scoring.dna_dihedral_bb, 1.0)
scorefxn.set_weight(rosetta.core.scoring.dna_dihedral_chi, 1.0)
scorefxn.set_weight(rosetta.core.scoring.dna_dihedral_sugar, 0.1)

# declare DOFs we want to allow to minimize
mm = MoveMap()
mm.set_bb(True)
mm.set_chi(True)

pose.dump_pdb("/Users/tgoel/Downloads/1bna_ros.pdb")
min_pose(pose,scorefxn,mm)
pose.dump_pdb("/Users/tgoel/Downloads/1bna_ros_min.pdb")

core.import_pose.import_pose: File '1BNA.clean.pdb' automatically determined to be of type PDB
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: beta_genpot.wts
basic.io.database: Database file opened: scoring/dna/bound_dna_dihedrals.txt
core.energy_methods.GenericBondedEnergy: Creating new peptide-bonded energy container (24)
core.energy_methods.DNA_DihedralEnergy: Creating new peptide-bonded energy container (24)
core.optimization.LineMinimizer: Inaccurate G in step= 4.76837e-07 Deriv= -0.0085179 Finite Diff= 0.00190303


True